In [30]:
# standard imports
import os
import sys

# third party modules
import numpy as np
import pandas as pd
#import seaborn as sns
from ipywidgets import interactive
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [14, 7]
plt.rcParams.update({'font.size': 18})


# my custom coded modules
sys.path.append(os.path.join(os.getcwd(), "src"))
import file_handling as fh

# tensorflow is difficult
sys.path.append("/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages")
import tensorflow as tf
from tensorflow import keras

In [36]:
def interactive_plot(location=0):
    '''
    Creates two plots centered around the given location 
      - The six frame coding entropy
      - The 'true' annotated frames
    '''
    fig, ax = plt.subplots(2)

    # plot the six frame entropy
    ax[0].scatter([n*3 for n in range(0,len(testcaseP[0::6]))], 
                  [1 if item==2 else None for item in testcaseP[0::6]], color = 'r',
                 )
    ax[0].scatter([n*3 for n in range(0,len(testcaseP[1::6]))],
                  [-1 if item==2 else None for item in testcaseP[1::6]], color = 'r',
                 )
    ax[0].scatter([n*3 for n in range(0,len(testcaseP[2::6]))], 
                  [2 if item==2 else None for item in testcaseP[2::6]], color = 'r',
                 )
    ax[0].scatter([n*3 for n in range(0,len(testcaseP[3::6]))],
                  [-2 if item==2 else None for item in testcaseP[3::6]], color = 'r',
                 )
    ax[0].scatter([n*3 for n in range(0,len(testcaseP[4::6]))],
                  [3 if item==2 else None for item in testcaseP[4::6]], color = 'r',
                 )
    ax[0].scatter([n*3 for n in range(0,len(testcaseP[5::6]))],
                  [-3 if item==2 else None for item in testcaseP[5::6]], color = 'r',
                 )
                   
    ax[0].axhline(y=0, color='grey', linestyle='-.')
    ax[0].set_yticks([1,2,3,-1,-2,-3])

    ax[0].set_ylabel('Predicted')

    #plot the annotated 'true' frames
    ax[1].scatter(list(annotated_frames.keys()), list(annotated_frames.values()), color='blue')
    
    ax[1].axhline(y=0, color='grey', linestyle='-.')
    ax[1].set_yticks([1,2,3,-1,-2,-3])
    ax[1].set_ylabel('Actual')

    ax[0].title.set_text('Using aminoacid frequencies in a sliding window for each frame to predict coding/noncoding')
    ax[1].set_xlabel('Position Along Genome')
    ax[0].set_xlim(location, location+3000)
    ax[1].set_xlim(location, location+3000)

def create_model(opt):
    '''
    This creates and returns a new model
    '''
    model = keras.Sequential([
        keras.layers.Dense(15, activation='relu', input_shape=(22,)),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer = opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy']
             )
    return model

In [32]:
# (this step takes about a minute)

# create a new model
model = create_model('Adam')
# load the weights from the previous model
model.load_weights("cp.ckpt")

# load a single genome and then predict the coding types of the frames 
testcase = pd.read_csv('1007869.8.tsv.gz', compression='gzip', header=0, sep='\t', dtype={'FILENAME': 'str'})
testcaseX, testcaseY = testcase.loc[:,'A':'V'], testcase['TYPE'].replace({'i':0, 'n':1, 'c':2})


In [34]:
# here is what the data looks like, it is just the 40 codon/aminoacid window at each 
# base position, forward and reverse, in the ~142k genome
# So row 0 is frame 1, which is (c)oding, row 1 is frame -1, which is (n)oncoding
# row 2 is frame 2, row 3 is frame -2, etc
# The columns A-V are the aminoacid counts, and rows (that are not ends) sum to 40
# The AT skew is not used for anything
testcase

,FILENAME,AT_SKEW,TYPE,A,R,N,D,B,C,E,...,L,K,M,F,P,S,T,W,Y,V
0,1007869.8,0.33,c,3,4,0,0,0,0,2,...,0,0,1,0,0,4,0,1,0,2
1,1007869.8,0.33,n,1,0,1,0,0,0,0,...,3,0,1,0,3,4,2,0,0,0
2,1007869.8,0.33,n,1,1,1,0,0,0,2,...,3,0,0,0,2,1,0,1,0,2
3,1007869.8,0.33,n,1,0,1,1,0,0,1,...,3,0,0,0,4,1,1,1,0,0
4,1007869.8,0.33,n,1,1,0,1,0,2,0,...,0,1,1,1,2,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285117,1007869.8,0.33,i,1,1,0,0,0,0,0,...,0,0,0,0,2,0,4,3,1,0
285118,1007869.8,0.33,i,1,0,0,0,0,0,0,...,1,0,0,0,8,0,0,0,1,4
285119,1007869.8,0.33,i,0,3,0,2,0,0,0,...,0,0,1,0,0,0,2,2,1,3
285120,1007869.8,0.33,i,0,2,0,0,0,1,0,...,1,0,0,0,4,2,3,0,1,2


In [37]:
model_predictions = model.predict(testcaseX)

testcaseP = np.argmax(model_predictions,axis=-1)

# plot the predicted frames versus the actual frames
annotated_frames = fh.read_gff("1007869.8.gff.gz")   
interactive(interactive_plot, location=(0,len(testcase),1000))

interactive(children=(IntSlider(value=0, description='location', max=285122, step=1000), Output()), _dom_class…